In [1]:
# First approach to Smart Charging Phase 1 using PuLP.
# Modeled as a PuLP optimisation blending problem
# Started 20 Aug 2020
# Author: Sofia Taylor, Flexible Power Systems
# test new laptop

import numpy as np
import pandas as pd
import datetime as dt
from pulp import *
import pickle
import global_variables as gv
import functions as f
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import timeit
import json 
import time
import random
import matplotlib.dates as mdates
import matplotlib.font_manager as font_manager
import lin_prog_functions as lpf


In [2]:
empty_profile = pickle.load(open('Outputs/empty_profile','rb'))
journeys = pickle.load(open('Outputs/journeys_range','rb'))
dates = np.unique(empty_profile.index.get_level_values(0).date)
dates = np.delete(dates, [1,2,3,4,5])
dates


array([datetime.date(2019, 2, 10), datetime.date(2019, 2, 16),
       datetime.date(2019, 2, 17)], dtype=object)

In [3]:
all_days_profile = []
status = 0
i = 0
for date in dates:
    start = time.process_time()
    day = dt.datetime.combine(date, dt.datetime.min.time())
    day_journeys = f.get_daily_data(journeys, day)
    day_profile = f.create_daily_schedule(empty_profile, day)
    if len(day_profile)==0:
        print('Empty day',day)
        pass
    else:
        output_df = {}
        PuLP_prob = {}
        day_profile_out = day_profile.copy()
        for ca in gv.CATS:
            output_df[ca], PuLP_prob[ca] = lpf.linear_optimiser_V1(
                day_profile,
                day_journeys,
                ca
                )
            day_profile_out = day_profile_out.merge(
            output_df[ca],
            how='left',
            left_index=True,
            right_index=True,
            )
        
        print(
            date,
            '\nTime:', time.process_time() - start,
            '\nStatus:',PuLP_prob['opt'].status, PuLP_prob['BAU'].status,PuLP_prob['BAU2'].status,
            '\nCost:', value(PuLP_prob['opt'].objective))
        all_days_profile.append(day_profile_out)
df = pd.concat(all_days_profile)

2019-02-10 
Time: 9.171875 
Status: 1 1 1 
Cost: 8305.797331467598
2019-02-16 
Time: 4.5625 
Status: 1 1 1 
Cost: 4355.325559858401
Empty day 2019-02-17 00:00:00


In [9]:
journeys.index.get_level_values(1)

Int64Index([2104108, 2104116, 2104120, 2104123, 2104125, 2104126, 2104130,
            2104134, 2104137, 2104142,
            ...
            2112914, 2112917, 2112919, 2112921, 2112923, 2112928, 2112929,
            2112933, 2112936, 2112940],
           dtype='int64', name='Route_ID', length=257)

In [29]:
# Creates summary columns and dataframes from outputs
def summary_outputs(profile, journeys):
    cols=gv.CAT_COLS
    routes = journeys.index.get_level_values(1)
    range_profile = profile.copy()
    range_journeys = journeys.copy()
    for ca in gv.CATS:
        range_profile[cols['CHARGE_DEL'][ca]] = (
            range_profile[cols['OUTPUT'][ca]] 
            * gv.CHARGER_EFF)
        range_profile[cols['ECOST'][ca]] = (
            range_profile[cols['OUTPUT'][ca]] 
            * range_profile[cols['PRICE']['opt']])
        # Remove SOC from here, add to optimiser. Add a reasonable SoC for the vehicle summary
        # for route in routes:
            # opt = (gv.BATTERY_CAPACITY - journeys.loc[(slice(None),route),'Energy_Required'] 
            # + range_profile.loc[(slice(None),route),cols['CHARGE_DEL'][ca]].cumsum())*100 / gv.BATTERY_CAPACITY
            # range_profile.loc[(slice(None),route),cols['SOC'][ca]] = opt

    #     day_journeys[cols['OUTPUT'][ca]] = day_profile[cols['OUTPUT'][ca]].groupby(level=1).sum()
    #     day_journeys[cols['ECOST'][ca]] = day_profile[cols['ECOST'][ca]].groupby(level=1).sum()
    #     day_journeys[cols['PEAK'][ca]] = day_profile[cols['OUTPUT'][ca]].groupby(level=1).max()

    # site = day_profile.groupby(level=0).sum()
    # site[cols['PRICE']['opt']] = day_profile[cols['PRICE']['opt']].groupby(level=0).mean()
    # for ca in gv.CATS:
    #     site[cols['SOC'][ca]] = day_profile[cols['SOC'][ca]].groupby(level=0).mean()
    #     site[cols['NUM'][ca]] = day_profile[cols['OUTPUT'][ca]].astype(bool).groupby(level=0).sum()
    # site.drop(columns=[cols['PRICE']['BAU'], 'Vehicle_ID'], inplace=True)

    # global_summary = site.sum()
    # global_summary.drop(labels=[cols['PRICE']['opt']], inplace=True)
    # for ca in gv.CATS:
    #     global_summary.drop(labels=[cols['SOC'][ca],cols['NUM'][ca]], inplace=True)
    return range_profile#, day_journeys, site, global_summary

In [31]:
range_profile =summary_outputs(df, journeys)


(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)
(48,)
(1,)


KeyError: 2106398